## 1. Build and train a keras sequential model to classify digits form the mnist dataset. The model must have a hidden dense layer of 128 neurons with a relu activation function.

In [3]:
import tensorflow

In [4]:
# Importa las bibliotecas necesarias
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt
import numpy as np

# Carga el conjunto de datos MNIST
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# Normaliza los datos de entrada
train_images = train_images.reshape((60000, 28 * 28))
train_images = train_images.astype('float32') / 255

test_images = test_images.reshape((10000, 28 * 28))
test_images = test_images.astype('float32') / 255

# Convierte las etiquetas a categorías
train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

# Construye el modelo secuencial
model = Sequential()
model.add(Dense(128, activation='relu', input_shape=(28 * 28,)))
model.add(Dense(10, activation='softmax'))

# Compila el modelo
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Entrena el modelo
model.fit(train_images, train_labels, epochs=5, batch_size=128)

# Evalúa el modelo
test_loss, test_acc = model.evaluate(test_images, test_labels)
print('Test accuracy:', test_acc)


Epoch 1/5
469/469 [==============================] - 1s 2ms/step - loss: 0.3577 - accuracy: 0.9019
Epoch 2/5
469/469 [==============================] - 1s 2ms/step - loss: 0.1615 - accuracy: 0.9527
Epoch 3/5
469/469 [==============================] - 1s 2ms/step - loss: 0.1177 - accuracy: 0.9662
Epoch 4/5
469/469 [==============================] - 1s 2ms/step - loss: 0.0919 - accuracy: 0.9738
Epoch 5/5
313/313 [==============================] - 0s 1ms/step - loss: 0.0891 - accuracy: 0.9738
Test accuracy: 0.973800003528595


In [ ]:

# Selecciona una imagen de prueba
image_index = 0
img = test_images[image_index]
plt.imshow(img.reshape(28, 28), cmap='gray')

# Predecir la etiqueta de la imagen de prueba
img = (np.expand_dims(img,0))
predictions_single = model.predict(img)
predicted_label = np.argmax(predictions_single)

# Imprimir la etiqueta real y la predicha
print("Etiqueta real: ", np.argmax(test_labels[image_index]))
print("Etiqueta predicha: ", predicted_label)

## 2.Now build and train a keras functional model for the same problem. The model must have a hidden dense layer of 128 neurons with an activation function defined by the following function: activation(x) = ( 0 if x < 0, sin(x −π/2) + 1 if 0 ≤ x ≤ π/2, x −π/2 + 1 otherwise )

In [5]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.utils import to_categorical
import numpy as np

# Define la función de activación personalizada
def custom_activation(x):
    cond = tf.less(x, 0)
    out1 = tf.zeros(tf.shape(x))
    cond2 = tf.less_equal(x, np.pi/2)
    out2 = tf.sin(x - np.pi/2) + 1
    cond3 = tf.greater(x, np.pi/2)
    out3 = x - np.pi/2 + 1
    return tf.where(cond, out1, tf.where(cond2, out2, out3))

# Carga el conjunto de datos MNIST
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# Normaliza los datos de entrada
train_images = train_images.reshape((60000, 28 * 28))
train_images = train_images.astype('float32') / 255

test_images = test_images.reshape((10000, 28 * 28))
test_images = test_images.astype('float32') / 255

# Convierte las etiquetas a categorías
train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

# Construye el modelo secuencial
model = Sequential()
model.add(Dense(128, input_shape=(28 * 28,)))
model.add(Activation(custom_activation))
model.add(Dense(10, activation='softmax'))

# Compila el modelo
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Entrena el modelo
model.fit(train_images, train_labels, epochs=5, batch_size=128)

# Evalúa el modelo
test_loss, test_acc = model.evaluate(test_images, test_labels)
print('Test accuracy:', test_acc)

Epoch 1/5
469/469 [==============================] - 2s 3ms/step - loss: 0.3696 - accuracy: 0.9014
Epoch 2/5
469/469 [==============================] - 1s 3ms/step - loss: 0.1653 - accuracy: 0.9531
Epoch 3/5
469/469 [==============================] - 1s 2ms/step - loss: 0.1202 - accuracy: 0.9656
Epoch 4/5
469/469 [==============================] - 1s 2ms/step - loss: 0.0956 - accuracy: 0.9725
Epoch 5/5
313/313 [==============================] - 0s 928us/step - loss: 0.1030 - accuracy: 0.9686
Test accuracy: 0.9685999751091003


## 3. Repeat the previous questions but now using PyTorch.

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms

# Define la función de activación personalizada
class CustomActivation(nn.Module):
    def forward(self, x):
        cond = x < 0
        out1 = torch.zeros_like(x)
        cond2 = (0 <= x) & (x <= np.pi/2)
        out2 = torch.sin(x - np.pi/2) + 1
        cond3 = x > np.pi/2
        out3 = x - np.pi/2 + 1
        return torch.where(cond, out1, torch.where(cond2, out2, out3))

# Define el modelo
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(28*28, 128)
        self.fc2 = nn.Linear(128, 10)
        self.custom_activation = CustomActivation()

    def forward(self, x):
        x = x.view(-1, 28*28)
        x = self.custom_activation(self.fc1(x))
        x = F.softmax(self.fc2(x), dim=1)
        return x

# Carga el conjunto de datos MNIST
train_data = datasets.MNIST(root='data', train=True, download=True, transform=transforms.ToTensor())
test_data = datasets.MNIST(root='data', train=False, download=True, transform=transforms.ToTensor())

train_loader = torch.utils.data.DataLoader(train_data, batch_size=128, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=128, shuffle=True)

# Instancia el modelo, define la pérdida y el optimizador
model = Net()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

# Entrena el modelo
for epoch in range(5):
    for data, target in train_loader:
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

# Evalúa el modelo
correct = 0
total = 0
with torch.no_grad():
    for data, target in test_loader:
        output = model(data)
        _, predicted = torch.max(output.data, 1)
        total += target.size(0)
        correct += (predicted == target).sum().item()

print('Test accuracy:', correct / total)